# How to do Periodic Boundary Conditions

It's easy. We only need to tag the boundary as 'p', and dont put a wall in $(x y)$ boundaries.

In [1]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import string as st


from IPython.display import HTML

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
radius = 1.4 #um
frequency = 1 #Hz
angle = 27 #degrees
region, initial_positions = initial_setup(n_of_particles = 150, packing=0.3, height = 4)

sim = lmp.sim(initial_positions,
          magnitude = 7.28,
          file_name = "periodic_system_explore",
          dir_name = "PeriodicB",
          radius = radius,
          diffusion = 0.14,
          frequency=frequency,
          susceptibility = 0.4,
          angle=angle,
          framerate=1e1,
          timestep=1e-4,
          total_time = 20,
          temperature = 300,
          stamp_time = False,
          space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

sim.generate_scripts()
sim.run()

HTML(direct_display(sim))

# Support Functions

In [3]:
def initial_setup(n_of_particles, packing = 0.3, height = 4, radius=1.4):

    part_in_edge = np.round(np.sqrt(n_of_particles))
    n_of_particles = part_in_edge**2

    area_particle = n_of_particles*radius**2*np.pi
    area_region = area_particle/packing

    length_region = np.sqrt(area_region)
    x_loc = np.linspace(-length_region/2+radius,length_region/2-radius,part_in_edge)
    y_loc = np.linspace(-length_region/2+radius,length_region/2-radius,part_in_edge)

    [X,Y] = np.meshgrid(x_loc,y_loc)
    Z = np.zeros(np.shape(X))

    initial_positions = np.array([[x,y,z] for (x,y,z) in zip(X.flatten(),Y.flatten(),Z.flatten())])
    
    if np.mean(np.diff(x_loc))<2*radius:
        raise ValueError("packing is too high")

    region = [np.round(length_region),np.round(length_region),height]
    return region, initial_positions

def animate_trj(trj,sim,ax=False):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    idx = pd.IndexSlice
    
    if not ax:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    particles = trj.index.get_level_values('id').unique()
    n_of_particles = len(trj.index.get_level_values('id').unique())
    
    region = sim.sim_parameters.space["region"] 
    radius = sim.particle_properties[0].radius
    
    framerate = sim.run_parameters.framerate
    runtime = sim.run_parameters.total_time
    timestep = sim.run_parameters.timestep
    
    lammps_time = 1e6;
    
    dt_data = np.round(1/(timestep*framerate)) # Data timestep in lammps_time
    dt_video = 1/framerate*1000 # video timestep in miliseconds
    frames = runtime*framerate+1
    
    ax.set_xlim(region[0],region[1])
    ax.set_ylim(region[2],region[3])
    ax.set(aspect='equal')
    ax.set_xlabel("$x [\mu{m}]$")
    ax.set_ylabel("$y [\mu{m}]$")
    
    patches = []
    for i,p in enumerate(particles):
        c = plt.Circle((0, 0), radius)
        patches.append(c)

    p = clt.PatchCollection(patches, cmap=plt.cm.RdBu)
    p.set_array(np.zeros(0))
    p.set_clim([region[4]+radius,region[5]-radius])
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0)
    plt.colorbar(p,label='$z [\mu{m}]$',cax=cax)

    def init():
        ax.add_collection(p)
        return p,

    def animate(frame):

        for (part_id,particle) in enumerate(particles):
            patches[part_id].center = (trj.loc[idx[frame*dt_data,particle],'x'],trj.loc[idx[frame*dt_data,particle],'y'])
        p.set_paths(patches)
        p.set_array(trj.loc[idx[frame*dt_data,:],'z'].values)
        ax.add_collection(p)
        return p,

    anim = anm.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(frames), interval=dt_video, blit=True);
    plt.close(anim._fig)

    return anim

def display(sim):
    trj = sim.load(read_trj=True)
    anim = animate_trj(trj,sim)
    anim.save(sim.base_name+".gif",writer = "imagemagick")
    video_html = st.Template(""" <video controls>
          <source src="$name" type="video/mp4">
            </video> """).substitute(name=sim.base_name+".mp4")
    HTML(video_html)
    return anim

def direct_display(sim):
    trj = sim.load(read_trj=True)
    anim = animate_trj(trj,sim)
    return anim.to_html5_video()